In [1]:
import os

import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import seaborn as sns
from util import *
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
instances = pd.read_pickle('dataset/labelenc/crossvalidation/X_train.pkl')
labels = pd.read_pickle('dataset/labelenc/crossvalidation/y_train.pkl')

In [3]:
X_train = pd.read_pickle('dataset/labelenc/split/X_train.pkl')
X_test = pd.read_pickle('dataset/labelenc/split/X_test.pkl')
y_train = pd.read_pickle('dataset/labelenc/split/y_train.pkl')
y_test = pd.read_pickle('dataset/labelenc/split/y_test.pkl')

# Random search

In [4]:
params = {
 'learning_rate' : [0.05,0.10,0.15],
 'max_depth' : [ 6, 7, 8, 9, 10],
 'gamma': [ 0.1, 0.2 , 0.3, 0.4 ],
}

In [5]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost
classifier = xgboost.XGBClassifier()

In [6]:
rs_model=RandomizedSearchCV(classifier,param_distributions=params,scoring='roc_auc',n_iter = 2, cv=5,verbose=3)

In [7]:
rs_model.fit(instances, labels)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END gamma=0.2, learning_rate=0.15, max_depth=9;, score=0.943 total time= 6.3min
[CV 2/5] END gamma=0.2, learning_rate=0.15, max_depth=9;, score=1.000 total time= 6.0min
[CV 3/5] END gamma=0.2, learning_rate=0.15, max_depth=9;, score=1.000 total time= 6.1min
[CV 4/5] END gamma=0.2, learning_rate=0.15, max_depth=9;, score=1.000 total time= 6.2min
[CV 5/5] END gamma=0.2, learning_rate=0.15, max_depth=9;, score=1.000 total time= 6.1min
[CV 1/5] END gamma=0.2, learning_rate=0.05, max_depth=7;, score=0.945 total time= 4.8min
[CV 2/5] END gamma=0.2, learning_rate=0.05, max_depth=7;, score=0.995 total time= 4.5min
[CV 3/5] END gamma=0.2, learning_rate=0.05, max_depth=7;, score=0.995 total time= 4.8min
[CV 4/5] END gamma=0.2, learning_rate=0.05, max_depth=7;, score=0.995 total time= 4.6min
[CV 5/5] END gamma=0.2, learning_rate=0.05, max_depth=7;, score=0.995 total time= 4.6min


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None,
                                           reg_alpha=None, reg_lambda=None, ...),
                   n_iter=2,
                   param_distributions={'gamma': [0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15],
                                        'max_depth': [6, 7, 8, 9, 10]},
                   scoring='roc_auc', verbose=3)

In [8]:
rs_model.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

# Model (with Random search)

In [20]:
clf = xgb.XGBClassifier(
    gamma = 0.2,
    n_estimators=500,
    max_depth=9,
    learning_rate=0.15,
    subsample=0.9,
    colsample_bytree=0.9,
    missing=-999,
    random_state=2019,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [21]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0.2, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
              missing=-999, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=2019, reg_alpha=0, reg_lambda=1, ...)

In [22]:
y_pred = clf.predict(X_test)

In [25]:
import pickle
pickle.dump(clf, open('xgboostModel.p', 'wb'))